In [14]:
%cd '/kaggle/input/urdu-comments-classification'

/kaggle/input/urdu-comments-classification


In [2]:
import pandas as pd

In [4]:
train = pd.read_excel("train.xlsx")
test = pd.read_excel("test.xlsx")
valid = pd.read_excel("valid.xlsx")

In [6]:
train.head()

,comment,target
0,متفق افراد کو جوڑنا سب کو ضرورت کی جگہ پر کر...,neu
1,شاھد بایھی اپ کو ون ڈےکرکٹ میں واپس آجاے,neu
2,سر مجھے بھی ایک دے دیں,neu
3,شاہد بھاہی جنرل راحیل کی طرح شیر ہے,neu
4,پوں پوں آفریدی,neu


In [7]:
test.head()

,comment,target
0,زمان خان نے آوٹ نہ کر کے اچھا کیا باقی سب گیم ...,neu
1,اس غریب کے پاس کوئئ سفارش نہیں تبھی اس بیچارے ...,neu
2,پاکستانی ٹیم میں کمزور پلیئر امام الحق محمد نو...,neu
3,اچھا تھمب نیل میں لڑکی دکھانا لازمی ہے کیا بات ہے,neu
4,آپ میری تجویز پر عمل کریں گے مجھے بڑی امید ھے,neu


In [8]:
valid.head()

,comment,target
0,کچے کیلے یہاں ناروے میں عام دکانوں سے کٹی حالت...,neu
1,کہیں سے بھی ملیں کیلے بس مل جائیں میں ریکوئسٹ ...,neu
2,آپ نے بھی سعود بھائی کی ترکیب سے بنائے تھے جاسمن؟,neu
3,کٹے ہوئے خشک کیلے پکوڑوں کی اس ترکیب کے لیے من...,neu
4,سعود بھائی اگر آپ وعدہ کریں کہ اسی طرح تفصیل س...,neu


In [9]:
print(train.shape,
test.shape,
valid.shape)

(6399, 2) (2001, 2) (1602, 2)


In [10]:
train = train.drop_duplicates()
test = test.drop_duplicates()
valid = valid.drop_duplicates()

In [11]:
print(train.shape,
test.shape,
valid.shape)

(6335, 2) (1984, 2) (1593, 2)


In [12]:
!pip install urduhack

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.5/105.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 73.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.5 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.1.5
    Uninstalling typeguard-4.1.5:
      Successfully uninstalled typeguard-4.1.5
  Attempting uninstall: keras
    Found existing installation: keras 3.3.3
    Uninstalling keras-3.3.3:
      Successfully uninstalled keras-3.3.3
  Attempting uninstall: Click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7
  Attempting uninstall: tensorflow-datasets
    Found existing installation: tensorflow-datasets 4.9.4
    Uninstalling tensorflow-datasets-4

In [17]:
!pip install demoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.7 MB/s eta 0:00:00


In [18]:
import demoji
import re

from urduhack.preprocessing import replace_urls
from urduhack.preprocessing import replace_numbers
from urduhack.preprocessing import remove_punctuation
from urduhack.preprocessing import remove_accents
from urduhack.preprocessing import remove_english_alphabets
from urduhack.preprocessing import normalize_whitespace
from urduhack.normalization import normalize

def clean_comment(comment):

    comment = demoji.replace(comment,"")

    comment = re.sub(r'@[A-Za-z0-9]+', '', comment)

    comment = re.sub(r'#[A-Za-z0-9]*', '', comment)

    comment = replace_urls(comment,replace_with="")

    comment = replace_numbers(comment,replace_with="")

    comment = remove_accents(comment)

    comment = remove_english_alphabets(comment)

    comment = normalize_whitespace(comment)

    comment = remove_punctuation(comment)

    comment = normalize(comment)

    comment = re.sub(r'\+', '', comment)

    comment = re.sub(r'\=', '', comment)

    return comment

2024-06-30 10:45:57.793013: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-30 10:45:57.793121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-30 10:45:57.924670: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our Tens

In [19]:
for i in train.index:
    a = train['comment'][i]
    b = clean_comment(a)
    train.loc[[i],['comment']] = b

for i in valid.index:
    a = valid['comment'][i]
    b = clean_comment(a)
    valid.loc[[i],['comment']] = b

for i in test.index:
    a = test['comment'][i]
    b = clean_comment(a)
    test.loc[[i],['comment']] = b

In [20]:
train.head()

,comment,target
0,متفق افراد کو جوڑنا سب کو ضرورت کی جگہ پر کرنا...,neu
1,شاھد بایھی اپ کو ون ڈےکرکٹ میں واپس آجاے,neu
2,سر مجھے بھی ایک دے دیں,neu
3,شاہد بھاہی جنرل راحیل کی طرح شیر ہے,neu
4,پوں پوں آفریدی,neu


In [21]:
stop_words = set("""

 آ آئی آئیں آئے آتا آتی آتے آداب آدھ آدھا آدھی آدھے آس آمدید آنا آنسہ آنی آنے
 آپ آگے آہ آہا آیا اب ابھی ابے اتوار ارب اربویں ارے اس اسکا اسکی اسکے اسی اسے اف
 افوہ الاول البتہ الثانی الحرام السلام الف المکرم ان اندر انکا انکی انکے انہوں انہی انہیں اوئے اور
 اوپر اوہو اپ اپنا اپنوں اپنی اپنے اپنےآپ اکبر اکثر اگر اگرچہ اگست اہاہا ایسا ایسی ایسے ایک بائیں
 بار بارے بالکل باوجود باہر بج بجے بخیر برسات بشرطیکہ بعد بعض بغیر بلکہ بن بنا بناؤ بند
 بڑی بھر بھریں بھی بہار بہت بہتر بیگم تاکہ تاہم تب تجھ تجھی تجھے ترا تری تلک تم تمام
 تمہارا تمہاروں تمہاری تمہارے تمہیں تو تک تھا تھی تھیں تھے تہائی تیرا تیری تیرے تین جا جاؤ
 جائیں جائے جاتا جاتی جاتے جانی جانے جب جبکہ جدھر جس جسے جن جناب جنہوں جنہیں جو جہاں جی
 جیسا جیسوں جیسی جیسے جیٹھ حالانکہ حالاں حصہ حضرت خاطر خالی خدا خزاں خواہ خوب خود دائیں درمیان
 دریں دو دوران دوسرا دوسروں دوسری دوشنبہ دوں دکھائیں دگنا دی دیئے دیا دیتا دیتی دیتے دیر دینا دینی
 دینے دیکھو دیں دیے دے ذریعے رکھا رکھتا رکھتی رکھتے رکھنا رکھنی رکھنے رکھو رکھی رکھے رہ رہا
 رہتا رہتی رہتے رہنا رہنی رہنے رہو رہی رہیں رہے ساتھ سامنے ساڑھے سب سبھی سراسر سلام سمیت سوا
 سوائے سکا سکتا سکتے سہ سہی سی سے شام شاید شکریہ صاحب صاحبہ صرف ضرور طرح طرف طور
 علاوہ عین فروری فقط فلاں فی قبل قطا لئے لائی لائے لاتا لاتی لاتے لانا لانی لانے لایا لو
 لوجی لوگوں لگ لگا لگتا لگتی لگی لگیں لگے لہذا لی لیا لیتا لیتی لیتے لیکن لیں لیے
 لے ماسوا مت مجھ مجھی مجھے محترم محترمہ محترمی محض مرا مرحبا مری مرے مزید مس مسز مسٹر مطابق
 مطلق مل منٹ منٹوں مکرمی مگر مگھر مہربانی میرا میروں میری میرے میں نا نزدیک نما نو نومبر
 نہ نہیں نیز نیچے نے و وار واسطے واقعی والا والوں والی والے واہ وجہ ورنہ وعلیکم وغیرہ ولے
 وگرنہ وہ وہاں وہی وہیں ویسا ویسے ویں پاس پایا پر پس پلیز پون پونا پونی پونے پھاگن
 پھر پہ پہر پہلا پہلی پہلے پیر پیچھے چاہئے چاہتے چاہیئے چاہے چلا چلو چلیں چلے چناچہ چند چونکہ
 چوگنی چکی چکیں چکے چہارشنبہ چیت ڈالنا ڈالنی ڈالنے ڈالے کئے کا کاتک کاش کب کبھی کدھر کر
 کرتا کرتی کرتے کرم کرنا کرنے کرو کریں کرے کس کسی کسے کل کم کن کنہیں کو کوئی کون
 کونسا کونسے کچھ کہ کہا کہاں کہہ کہی کہیں کہے کی کیا کیسا کیسے کیونکر کیونکہ کیوں کیے
 کے گئی گئے گا گرما گرمی گنا گو گویا گھنٹا گھنٹوں گھنٹے گی گیا ہائیں ہائے ہاڑ ہاں ہر
 ہرچند ہرگز ہزار ہفتہ ہم ہمارا ہماری ہمارے ہمی ہمیں ہو ہوئی ہوئیں ہوئے ہوا ہوبہو ہوتا ہوتی
 ہوتیں ہوتے ہونا ہونگے ہونی ہونے ہوں ہی ہیلو ہیں ہے یا یات یعنی یک یہ یہاں یہی یہیں


""".split())

def remove_stopwords(text):
    filtered_words = [word for word in text.split() if word not in stop_words]
    return " ".join(filtered_words)

In [22]:
train["comment"] = train.comment.map(remove_stopwords)
valid["comment"] = valid.comment.map(remove_stopwords)
test["comment"] = test.comment.map(remove_stopwords)

In [23]:
train.head()

,comment,target
0,متفق افراد جوڑنا ضرورت جگہ میدان پرفیکٹ فیصلےک...,neu
1,شاھد بایھی ون ڈےکرکٹ واپس آجاے,neu
2,سر,neu
3,شاہد بھاہی جنرل راحیل شیر,neu
4,پوں پوں آفریدی,neu


In [24]:
import urduhack
urduhack.download()
from urduhack.models.lemmatizer import lemmatizer

def lemitizeStr(text):
    lemme_txt = ""
    temp = lemmatizer.lemma_lookup(text)
    for t in temp:
        lemme_txt += t[0] + " "

    return lemme_txt

89078/89078 [==============================] - 0s 0us/step


In [25]:
train["comment"] = train.comment.map(lemitizeStr)
valid["comment"] = valid.comment.map(lemitizeStr)
test["comment"] = test.comment.map(lemitizeStr)

In [26]:
train.head()

,comment,target
0,متفق افراد جوڑنا ضرورت جگہ میدان پرفیکٹ فیصلےک...,neu
1,شاھد بایھی ون ڈےکرکٹ واپس آجاے,neu
2,سر,neu
3,شاہد بھاہی جنرل راحیل شیر,neu
4,پوں پوں آفریدی,neu


In [27]:
train['target'].value_counts()

target
neg    2441
neu    2026
pos    1868
Name: count, dtype: int64

In [28]:
test['target'].value_counts()

target
neg    769
neu    632
pos    583
Name: count, dtype: int64

In [29]:
valid['target'].value_counts()

target
neg    616
neu    504
pos    473
Name: count, dtype: int64

In [30]:
from sklearn.preprocessing import LabelEncoder

class_label_encoder = LabelEncoder()

train['target_encoded'] = class_label_encoder.fit_transform(train['target'])
test['target_encoded'] = class_label_encoder.fit_transform(test['target'])
valid['target_encoded'] = class_label_encoder.fit_transform(valid['target'])

In [31]:
train['target_encoded'].value_counts()

target_encoded
0    2441
1    2026
2    1868
Name: count, dtype: int64

In [32]:
x_train = train["comment"]
x_test = test["comment"]
x_valid = valid["comment"]
y_train = train[["target_encoded"]]
y_test = test[["target_encoded"]]
y_valid = valid[["target_encoded"]]

In [33]:
df = pd.concat([train,valid,test])

In [34]:
df.shape

(9912, 3)

In [35]:
from collections import Counter

def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count

In [36]:
counter = counter_word(df.comment)

In [37]:
num_unique_words = len(counter)
num_unique_words

12362

In [38]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=num_unique_words)
tokenizer.fit_on_texts(x_train)

In [39]:
word_index = tokenizer.word_index

In [40]:
len(word_index)

9175

In [41]:
X_train_seq = tokenizer.texts_to_sequences(x_train)
X_valid_seq = tokenizer.texts_to_sequences(x_valid)
X_test_seq = tokenizer.texts_to_sequences(x_test)

In [42]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 20
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding="post")
X_valid_padded = pad_sequences(X_valid_seq, maxlen=max_length, padding="post")
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding="post")
X_train_padded.shape, X_valid_padded.shape, X_test_padded.shape

((6335, 20), (1593, 20), (1984, 20))

In [43]:
print(x_train.iloc[10])

آج ارجنٹینا میچ دیکھ ارجنٹینا جرمنی رکھوں ارجنٹینا برازیل ٹیم فاتح باقی انگلیڈ کھیل مایوس رونی صورتحال سنبھال 


In [44]:
print(X_train_seq[10])

[22, 1256, 3, 37, 1256, 491, 2759, 1256, 794, 2, 524, 149, 4055, 16, 664, 2114, 990, 2115]


In [45]:
print(X_train_padded[10])

[  22 1256    3   37 1256  491 2759 1256  794    2  524  149 4055   16
  664 2114  990 2115    0    0]


In [47]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

encoder = OneHotEncoder(sparse_output=False)

# Fit and transform the data
encoded_array = encoder.fit_transform(train[['target']])

# Create a DataFrame with the encoded data
train_encoded = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out())

# Fit and transform the data
encoded_array = encoder.transform(valid[['target']])  # Use transform instead of fit_transform

# Create a DataFrame with the encoded data
valid_encoded = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out())

# Fit and transform the data
encoded_array = encoder.transform(test[['target']])  # Use transform instead of fit_transform

# Create a DataFrame with the encoded data
test_encoded = pd.DataFrame(encoded_array, columns=encoder.get_feature_names_out())


/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [48]:
y_train, y_valid, y_test = train_encoded, valid_encoded, test_encoded

In [49]:
from gensim.models import KeyedVectors

word2vec = KeyedVectors.load_word2vec_format('/kaggle/input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin',  binary=True, limit=10 ** 5)

In [50]:
import numpy as np

embedding_dim = 300
weight_matrix = np.zeros((num_unique_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    try:
        embedding_vector = word2vec[word]
        weight_matrix[i] = embedding_vector
    except KeyError:
        weight_matrix[i] = np.random.uniform(-5, 5, embedding_dim)

In [56]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
# Create a Multilabel Naive Bayes classifier
classifier = MultiOutputClassifier(MultinomialNB())

# Define the hyperparameter grid for grid search
param_grid = {
    'estimator__alpha': [0.1, 0.5, 1.0],  # Smoothing parameter for MultinomialNB
    'estimator__fit_prior': [True, False],  # Use class prior probabilities or not for MultinomialNB
}

# Define the evaluation metrics for grid search
scoring = {
    'F1 Score': make_scorer(f1_score, average='weighted', zero_division=1)
}

grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring=scoring, cv=5, refit='F1 Score')
grid_search.fit(X_train_padded, y_train)

# Get the best hyperparameters and the corresponding model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(best_model)
# Evaluate the best model on the test set
predictions = best_model.predict(X_test_padded)

MultiOutputClassifier(estimator=MultinomialNB(alpha=0.1, fit_prior=False))


In [59]:
# Classification Report
class_report = classification_report(y_test, predictions, target_names=['Positive', "Negative", "Neutral"], zero_division=1)
print("\nClassification Report:\n", class_report)


Classification Report:
               precision    recall  f1-score   support

    Positive       0.49      0.33      0.40       769
    Negative       0.34      0.77      0.47       632
     Neutral       0.30      0.73      0.43       583

   micro avg       0.35      0.59      0.44      1984
   macro avg       0.38      0.61      0.43      1984
weighted avg       0.39      0.59      0.43      1984
 samples avg       0.37      0.59      0.43      1984



In [61]:
from sklearn.linear_model import LogisticRegression

# Create a multilabel classifier using Logistic Regression
classifier = MultiOutputClassifier(LogisticRegression(max_iter=20000,class_weight='balanced'))

# Define the hyperparameter grid for grid search
param_grid = {
    'estimator__C': [0.01, 0.1, 1.0, 10.0],  # Regularization parameter (smaller values for stronger regularization)
    'estimator__solver': ['liblinear', 'lbfgs']
}
# Define the evaluation metrics for grid search
scoring = {
    'F1 Score': make_scorer(f1_score, average='weighted', zero_division=1)
}

grid_search = GridSearchCV(estimator=classifier, param_grid=param_grid, scoring=scoring, cv=5, refit='F1 Score')
grid_search.fit(X_train_padded,y_train)

# Get the best hyperparameters and the corresponding model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print(best_model)

# Evaluate the best model on the test set
predictions = best_model.predict(X_test_padded)
test_labels_np = y_test.to_numpy()

MultiOutputClassifier(estimator=LogisticRegression(C=0.01,
                                                   class_weight='balanced',
                                                   max_iter=20000,
                                                   solver='liblinear'))


In [63]:
class_report = classification_report(test_labels_np, predictions, target_names=['Positive', "Negative", "Neutral"], zero_division=1)
print("\nClassification Report:\n", class_report)


Classification Report:
               precision    recall  f1-score   support

    Positive       0.48      0.43      0.46       769
    Negative       0.36      0.70      0.47       632
     Neutral       0.31      0.69      0.43       583

   micro avg       0.36      0.59      0.45      1984
   macro avg       0.38      0.61      0.45      1984
weighted avg       0.39      0.59      0.45      1984
 samples avg       0.37      0.59      0.45      1984



In [76]:
import warnings
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, GridSearchCV, cross_val_predict
from sklearn.metrics import classification_report

# Ignore warnings
warnings.filterwarnings("ignore")

# Define the parameter grid
param_grid = {
    'n_neighbors': [5, 10, 15, 20, 25, 30, 35],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}

# Initialize the KNeighborsClassifier
model = KNeighborsClassifier()

# Use GridSearchCV for hyperparameter tuning
kfold = KFold(n_splits=10, random_state=22, shuffle=True)
grid_search = GridSearchCV(model, param_grid, cv=kfold, scoring='f1_weighted', n_jobs=-1)

# Fit the model with training data
grid_search.fit(X_train_padded,y_train)

# Get the best estimator
best_knn_classifier = grid_search.best_estimator_

# Predict on the test data
dt_knn = best_knn_classifier.predict(X_test_padded)

# Perform cross-validation prediction
y_pred = cross_val_predict(best_knn_classifier, X_test_padded,y_test, cv=kfold)

# Convert test labels to numpy array
test_labels_np = y_test.to_numpy()

# Classification Report
class_report = classification_report(test_labels_np, y_pred, target_names=['Positive', 'Negative', 'Neutral'], zero_division=1)
print("\nClassification Report:\n", class_report)

# Print best parameters found by GridSearchCV
print("\nBest Parameters:\n", grid_search.best_params_)



Classification Report:
               precision    recall  f1-score   support

    Positive       0.46      0.34      0.39       769
    Negative       0.38      0.31      0.34       632
     Neutral       0.30      0.17      0.22       583

   micro avg       0.39      0.28      0.33      1984
   macro avg       0.38      0.27      0.32      1984
weighted avg       0.38      0.28      0.32      1984
 samples avg       0.56      0.28      0.28      1984


Best Parameters:
 {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}


In [79]:
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Ignore warnings
warnings.filterwarnings("ignore")

# Define the parameter grid
param_grid = {
    'estimator__n_estimators': [50, 100, 200],
    'estimator__max_depth': [None, 10, 20, 30],
    'estimator__min_samples_split': [2, 5, 10],
    'estimator__min_samples_leaf': [1, 2, 4],
    'estimator__max_features': ['sqrt', 'log2']
}

# Initialize the RandomForestClassifier inside OneVsRestClassifier
forestClassifier = OneVsRestClassifier(RandomForestClassifier(random_state=42), n_jobs=-1)

# Use GridSearchCV for hyperparameter tuning
grid_search = GridSearchCV(forestClassifier, param_grid, cv=5, scoring='f1_weighted', n_jobs=-1)

# Fit the model with training data
grid_search.fit(X_train_padded, y_train)

# Get the best estimator
best_forest_classifier = grid_search.best_estimator_

# Predict on the test data
forest_preds = best_forest_classifier.predict(X_test_padded)

# Convert test labels to numpy array
test_labels_np = y_test.to_numpy()

# Classification Report
class_report = classification_report(test_labels_np, forest_preds, target_names=['Positive', 'Negative', 'Neutral'], zero_division=1)
print("\nClassification Report:\n", class_report)

# Print best parameters found by GridSearchCV
print("\nBest Parameters:\n", grid_search.best_params_)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1609: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))
/opt/conda/lib/python3.10/site-


Classification Report:
               precision    recall  f1-score   support

    Positive       0.50      0.46      0.48       769
    Negative       0.44      0.19      0.26       632
     Neutral       0.43      0.26      0.33       583

   micro avg       0.47      0.32      0.38      1984
   macro avg       0.46      0.30      0.36      1984
weighted avg       0.46      0.32      0.37      1984
 samples avg       0.65      0.32      0.31      1984


Best Parameters:
 {'estimator__max_depth': None, 'estimator__max_features': 'sqrt', 'estimator__min_samples_leaf': 1, 'estimator__min_samples_split': 2, 'estimator__n_estimators': 100}


In [80]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Bidirectional, SimpleRNN, Activation, LSTM, GRU, Embedding, Bidirectional, InputLayer, Conv1D, GlobalMaxPooling1D, Flatten
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

batch_size=32

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words, embedding_dim, weights=[weight_matrix], input_length=max_length))

model.add(Bidirectional(LSTM(200, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(200, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(200)))
model.add(Dropout(0.3))


model.add(Dense(3, activation='softmax'))
model.summary()

loss = keras.losses.CategoricalCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(learning_rate=0.008)
model.compile(loss=loss, optimizer=optim)

model.fit(X_train_padded, y_train, validation_data = (X_valid_padded,y_valid),batch_size = batch_size, epochs=100, verbose=2)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 300)           3708600   
                                                                 
 bidirectional_3 (Bidirecti  (None, 20, 400)           801600    
 onal)                                                           
                                                                 
 dropout_3 (Dropout)         (None, 20, 400)           0         
                                                                 
 bidirectional_4 (Bidirecti  (None, 20, 400)           961600    
 onal)                                                           
                                                                 
 dropout_4 (Dropout)         (None, 20, 400)           0         
                                                                 
 bidirectional_5 (Bidirecti  (None, 400)              

In [81]:
from sklearn.metrics import classification_report

y_pred_probs = model.predict(X_test_padded)

# Convert predictions to class labels
y_pred = np.argmax(y_pred_probs, axis=-1)

# Convert test labels to class labels
y_true = np.argmax(y_test.to_numpy(), axis=-1)

# Calculating accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy}')

# Generating classification report
report = classification_report(y_true, y_pred)
print('Classification Report:')
print(report)

62/62 [==============================] - 4s 46ms/step
Accuracy: 0.4188508064516129
Classification Report:
              precision    recall  f1-score   support

           0       0.45      0.46      0.46       769
           1       0.39      0.41      0.40       632
           2       0.40      0.37      0.39       583

    accuracy                           0.42      1984
   macro avg       0.42      0.41      0.42      1984
weighted avg       0.42      0.42      0.42      1984

